# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read csv
city_df = pd.read_csv('output_data/cities.csv')
city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Lebu,0,CL,1581460323,77,-37.62,-73.65,62.58,8.03
1,1,Manaure,0,CO,1581460324,79,11.78,-72.44,78.06,26.42
2,2,Punta Arenas,75,CL,1581460183,71,-53.15,-70.92,57.20,6.93
3,3,Vaini,75,TO,1581460324,83,-21.20,-175.20,84.20,8.05
4,4,Bichura,100,RU,1581460324,82,50.59,107.60,5.00,4.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#build heat map
markers_loc = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
max_humidity = humidity.max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(markers_loc, weights = humidity,
                                 dissipating = False, max_intensity = max_humidity,point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
city_df["Max Temp"] = city_df["Max Temp"].astype(float)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#set up DataFrame parameters
vacation = city_df.loc[(city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 80) & (city_df["Cloudiness"] == 0), :]
vacation = vacation.dropna(how='any')
vacation["Hotel Name"] = ""
vacation.head(15)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,1,Manaure,0,CO,1581460324,79,11.78,-72.44,78.06,26.42,
13,13,Catuday,0,PH,1581460326,74,16.29,119.81,75.27,22.17,
29,29,Viedma,0,AR,1581460329,46,-40.81,-63.00,72.95,19.93,
52,52,Hermanus,0,ZA,1581460333,81,-34.42,19.23,72.00,7.00,
64,64,Cape Town,0,ZA,1581460087,68,-33.93,18.42,73.99,12.75,
80,80,Punta Alta,0,AR,1581460339,45,-38.88,-62.07,76.64,8.77,
110,110,Vila do Maio,0,CV,1581460345,41,15.13,-23.22,75.20,18.34,
204,204,Matara,0,LK,1581460236,86,5.95,80.54,72.48,5.26,
234,234,Bumba,0,CD,1581460371,42,2.18,22.47,73.42,1.59,
256,256,Moyale,0,KE,1581460376,47,3.52,39.06,77.31,3.87,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#set up hotel DataFrame

hotel_df =[]

#build loop
for index, row in vacation.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params = {"radius": 5000,
              "types": "hotel",
              "key": g_key,
              "location": f"{lat},{lng}"}

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url,params=params)
    response = response.json()
    
    try:
        hotel_df.append((response)['results'][1]['name'])

    except:
        hotel_df.append("")

vacation["Hotel Name"] = hotel_df
        
vacation.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,1,Manaure,0,CO,1581460324,79,11.78,-72.44,78.06,26.42,Colegio Urbana Mixta
13,13,Catuday,0,PH,1581460326,74,16.29,119.81,75.27,22.17,Cape Bolinao Lighthouse
29,29,Viedma,0,AR,1581460329,46,-40.81,-63.00,72.95,19.93,Hotel casa crespo
52,52,Hermanus,0,ZA,1581460333,81,-34.42,19.23,72.00,7.00,Misty Waves Boutique Hotel
64,64,Cape Town,0,ZA,1581460087,68,-33.93,18.42,73.99,12.75,Southern Sun Waterfront Cape Town


In [7]:
hotel_df

['Colegio Urbana Mixta',
 'Cape Bolinao Lighthouse',
 'Hotel casa crespo',
 'Misty Waves Boutique Hotel',
 'Southern Sun Waterfront Cape Town',
 'ACA AUTOMOVIL CLUB ARGENTINO',
 'Hotel Marilu',
 'Dutch Reformed Church, Matara',
 'Bumba Airport',
 'M-Pesa Shashamane Connections Ltd',
 'Wings For Humanity, Inc',
 'Broadwater Mariner Resort',
 'Faithful Guest Lodge',
 'dyd -mar Hotel',
 'Pacific Rendezvous Motel Resort',
 'Hotel Punta León',
 'Noble Computers',
 'Casa Dora',
 'Hotel Novotel Dakar']

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation.iterrows()]
locations = vacation[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))